# goal: rewrite a resume depending on the job profile

## notes and questions
- we can pass a list of tasks as `context` to a task. the task will then take into account the output of those tasks in its executionn it will not run until it has the output(s) from those tasks.

## main code

In [3]:
import warnings
warnings.filterwarnings('ignore')  # don't bother showing warnings

from crewai import Agent, Task, Crew

from IPython.display import Markdown, display

import os
from utils import get_openai_api_key, pretty_print_result, get_serper_api_key
openai_api_key = get_openai_api_key()
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [4]:
# tools

from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.37it/s]


In [5]:
display(Markdown("./fake_resume.md"))


# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, where he has excelled in leading both remote and in-office engineering teams. His expertise spans across software development, process innovation, and enhancing team collaboration. He is highly proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside deep expertise in various front end frameworks. Noah's significant experience in data science and machine learning has enabled him to spearhead successful deployments of scalable AI solutions and innovative data model development.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Noah has transformed the engineering division into a key revenue pillar for DataKernel, rapidly expanding the customer base from inception to a robust community.
- He spearheaded the integration of cutting-edge AI technologies and the use of scalable vector databases, which significantly enhanced the product's capabilities and market positioning.
- Under his leadership, the team has seen a substantial growth in skill development, with a focus on achieving strategic project goals that have significantly influenced the company's direction.
- Noah also played a critical role in defining the company’s long-term strategic initiatives, particularly in adopting AI technologies that have set new benchmarks within the industry.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed the engineering strategy and operations in close collaboration with C-level executives, playing a pivotal role in shaping the company's technological trajectory.
- Managed diverse teams across multiple time zones in North America and Europe, creating an environment of transparency and mutual respect which enhanced team performance and morale.
- His initiatives in recruiting, mentoring, and retaining top talent have been crucial in fostering a culture of continuous improvement and high performance.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Noah founded InnovPet, a startup focused on innovative IoT solutions for pet care, including a revolutionary GPS tracking collar that significantly enhanced pet safety and owner peace of mind.
- He was responsible for overseeing product development from concept through execution, working closely with engineering teams and marketing partners to ensure a successful market entry.
- Successfully set up an advisory board, established production facilities overseas, and navigated the company through a successful initial funding phase, showcasing his leadership and entrepreneurial acumen.
- Built the initial version of the product leveraging MongoDB

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Noah was instrumental in formulating and executing strategic plans that enhanced inter-departmental coordination and trust, leading to better project outcomes.
- He managed multiple engineering teams, fostering a culture that balances productivity with innovation, and implemented goal-setting frameworks that aligned with the company's long-term goals.
- Was a bery hands on manager using ruby on rails and react to build out a new product.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led the formation and development of a high-performance engineering team that was pivotal in increasing company revenue by 500% within two years.
- His leadership in integrating data analytics into business decision-making processes led to the development of a predictive modeling tool that revolutionized customer behavior analysis.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized a central API that significantly improved the functionality used by a large engineering team and thousands of users, enhancing overall system performance and user satisfaction.
- Implemented several critical enhancements, including advanced caching strategies that drastically reduced response times and system loads.

### BetCraft: CTO — 2013 - 2015
- Led the technology strategy post-Series A funding, directly reporting to the board and guiding the company through a phase of significant technological advancement and network expansion.
- His strategic initiatives and partnerships significantly improved platform performance and expanded the company's market reach.
- Helped build his initial product using both React and Angular and got pretty good at it.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah Williams is an ideal candidate for senior executive roles, particularly in companies seeking leadership with a robust blend of technical and strategic expertise.


In [6]:
# agents

# job researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)


# job applicant proflier
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)


# resume refiner
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)


# interview prep helper
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

In [8]:
# tasks

# extract job requriements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)


# compile comprehensive user profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)


# align resume with job requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)


# develop interview prep material
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],  # this task will use the output from these tasks for context
    agent=interview_preparer
)

In [10]:
# assembling the crew

job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],
    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],
    verbose=True
)

job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

result = job_application_crew.kickoff(inputs=job_application_inputs)

2025-03-03 13:25:50,995 - 8469596224 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

In [11]:
display(Markdown("./tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Full Stack Engineer with an 18-year tenure in the technology industry, proficient in front-end and back-end development, and skilled in multiple programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir. Noah has a proven track record of developing both desktop and mobile applications and managing well-functioning databases. His deep expertise in AI and data science has enabled him to lead successful deployments of scalable software solutions that ensure responsiveness and efficiency.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Transformed engineering practices to focus on building software solutions in collaboration with product managers.
- Led the development of user-centric applications with appealing visual design using modern front-end frameworks.
- Spearheaded the integration of cutting-edge AI technologies and scalable vector databases to enhance application functionality and user experience.
- Pioneered the implementation of robust security and data protection settings within the company’s software solutions.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Oversaw the development and management of databases and back-end applications, ensuring high performance and scalability.
- Facilitated technical discussions with engineering teams and business stakeholders, enhancing project delivery and strategic decision-making.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Developed a mobile application for innovative IoT solutions, focusing on user-friendly design and efficient data handling.
- Managed full-stack development from concept through execution, utilizing MongoDB, React, and custom APIs.

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Implemented new features and applications, significantly improving the product offerings of the company.
- Utilized Ruby on Rails and React to enhance front-end functionality and user interaction.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Focused on troubleshooting, debugging, and upgrading the company’s software products, leading to a marked increase in system reliability and performance.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized APIs and back-end services for high-load environments, contributing to the robustness and scalability of business applications.

### BetCraft: CTO — 2013 - 2015
- Led the technical strategy for the development of desktop and mobile applications using Angular and React, ensuring cross-platform functionality and excellent user experiences.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah Williams is an exemplary candidate for the Full Stack Engineer role at AI Fund, equipped with extensive technical skills, a strategic mindset, and a strong foundation in both front-end and back-end development. His leadership and technical abilities make him a perfect fit for driving innovative software solutions in a dynamic and growth-oriented environment.

In [12]:
display(Markdown("./interview_materials.md"))

**Interview Questions and Talking Points for Noah Williams for the Full Stack Engineer Position at AI Fund**

1. **Experience with Software Development:**
   - **Question:** "Can you discuss a project where you led the development from ideation to deployment, particularly focusing on how you managed both front-end and back-end aspects?"
   - **Talking Point:** Mention your role at DataKernel, emphasizing your strategic integration of AI technologies and scalable vector databases.

2. **Technical Skills in Programming:**
   - **Question:** "What are your preferred stacks for full-stack development and why? Can you provide examples of applications you've developed using these stacks?"
   - **Talking Point:** Draw on your extensive experience with Python, Ruby, and JavaScript, and discuss specific projects like InnovPet and your work at EliteDevs.

3. **Leadership in Software Engineering:**
   - **Question:** "How do you foster innovation and technical skill development among your engineering teams?"
   - **Talking Point:** Highlight your leadership at DataKernel and InnovPet, focusing on team skill growth and the adoption of new technologies.

4. **UI/UX Design Experience:**
   - **Question:** "Could you share your approach to creating user-centric designs and perhaps discuss a project where your input significantly enhanced the user experience?"
   - **Talking Point:** Reference your work at DataKernel, where you led the development of user-centric applications with appealing visual design.

5. **Database and API Management:**
   - **Question:** "Describe your experience in developing and managing databases and writing effective APIs. How do you ensure their efficiency and security?"
   - **Talking Point:** Talk about your hands-on experience at DriveAI and InnovPet, focusing on your strategic use of MongoDB and custom APIs.

6. **Problem Solving and Innovation:**
   - **Question:** "Can you provide an example of a challenging technical problem you solved? What was the impact of your solution?"
   - **Talking Point:** Discuss the orchestration framework CrewAI and its impact on collaborative intelligence among AI agents.

7. **Communication and Collaboration:**
   - **Question:** "How do you communicate complex technical issues to non-technical stakeholders?"
   - **Talking Point:** Utilize examples from your career where you effectively bridged the gap between technical teams and business stakeholders, particularly at DataKernel.

8. **Adaptability and Learning:**
   - **Question:** "The technology landscape is constantly evolving. How do you stay updated with the latest technologies and integrate them into your work?"
   - **Talking Point:** Mention your ongoing learning through certifications, such as the Data Science Specialization, and how you apply these learnings to real-world projects.

9. **Cultural Fit and Leadership Style:**
   - **Question:** "AI Fund values accountability, grit, and ownership. Can you provide examples from your career where these values were evident?"
   - **Talking Point:** Reflect on your experiences founding InnovPet and leading significant changes at DataKernel, demonstrating your commitment to these values.

10. **Future Vision and Goals:**
    - **Question:** "What are your professional goals for the next five years, and how does the role at AI Fund align with these goals?"
    - **Talking Point:** Discuss your vision for leveraging AI and data science in software development, aligning with AI Fund's focus on innovative software solutions.

These questions and talking points are designed to help Noah showcase his technical expertise, leadership skills, and alignment with the values and goals of AI Fund, positioning him as a strong candidate for the role.